In [ ]:
# default_exp changeEmail

## Change Email

> Change email of this account

In [ ]:
#hide
import src.setEnv 
import requests

In [ ]:
#export 
from src.userTable import UserTable
from awsSchema.apigateway import Response, Event
from copy import deepcopy
from src.saltHashPassword import hash_password, check_password
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from beartype import beartype

## Change Email Input Class

In [ ]:
#export 
@dataclass_json
@dataclass
class ChangeEmailInput:
    username: str
    password: str
    newEmail: str
    
    @property
    def passwordHash(self):
        return hash_password(self.newPassword)
    
    def checkPassword(self):
        for item in UserTable.username_index.query(self.username):
            return check_password(item.passwordHash, self.password)

## Main Lambda Handler

In [ ]:
#export
class H:
    class ParseInputError(Exception): pass
    class PasswordCheckError(Exception): pass
    class InvalidPasswordError(Exception): pass
    class ChangeEmailError(Exception): pass
    
    @classmethod
    @beartype
    def parseInput(cls, event: dict)->ChangeEmailInput:
        try:
            user = Event.parseDataClass(ChangeEmailInput, deepcopy(event))
            return user
        except Exception as e:
            raise cls.ParseInputError(e)
    
    @classmethod
    @beartype
    def changeToNewEmail(cls, user: ChangeEmailInput):
        try:
            passwordChecked = user.checkPassword()
        except Exception as e:
            raise cls.PasswordCheckError(e)
        if passwordChecked:
            pass
        else:
            raise cls.InvalidPasswordError
        try:
            usernameQuery = UserTable.username_index.query(user.username)
            for i in usernameQuery:
                userId = i.userId
                oldItem = UserTable.get(userId)
                oldItem.email = user.newEmail
                oldItem.save()        
        except Exception as e:
            raise cls.ChangeEmailError(e)
        return True

In [ ]:
#export 
def changeEmail(event, *args):
    try:
        user = H.parseInput(event)
        H.changeToNewEmail(user)
        return Response.returnSuccess()
    except H.ParseInputError as e:
        return Response.returnError(f'failed to parse input {e}')
    except H.PasswordCheckError as e:
        return Response.returnError(f'failed to check password {e}')
    except H.InvalidPasswordError as e:
        return Response.returnError(f'incorrect old password inputted {e}')
    except H.ChangeEmailError as e:
        return Response.returnError(f'unable to change password successfully {e}')
    except Exception as e:
        return Response.returnError(f' unknown error {e}')

### Success Test

In [ ]:
input_ = {'username': 'Delg45678', 'password': '12765', 'newPassword': '12764'}
event = Event.getInput(body = input_)
changeEmail(event)